In [1]:
%matplotlib inline
import torch as t
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [2]:
#############################################################生成数据集

#features是训练数据特征  labels是数据标签

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2

features = t.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=t.float)

labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b

labels += t.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=t.float)

In [3]:
#############################################################读取数据

###PyTorch提供了data包来读取数据。由于data常用作变量名，我们将导入的data模块用Data代替。
###在每一次迭代中，我们将随机读取包含10个数据样本的小批量。
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

##读取并打印第一个批量数据样本
for X, y in data_iter:
    print(X, "\n",y)
    break

tensor([[ 0.7830,  0.3203],
        [ 0.0744,  1.4822],
        [ 0.6173, -0.0365],
        [-0.2738, -0.9321],
        [-2.1059, -0.3814],
        [ 0.2378,  1.2565],
        [ 1.4830,  1.1565],
        [-1.4735,  0.9560],
        [ 1.5027,  2.8062],
        [ 0.1211,  1.0883]]) 
 tensor([ 4.6753, -0.6792,  5.5504,  6.8198,  1.2808,  0.4079,  3.2203, -2.0079,
        -2.3272,  0.7244])


In [4]:
#############################################################定义模型

#首先，导入torch.nn模块。实际上，“nn”是neural networks（神经网络）的缩写。顾名思义，该模块定义了大量神经网络的层。
#nn的核心数据结构是Module，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。
#在实际使用中，最常见的做法是继承nn.Module，撰写自己的网络层。


import torch.nn as nn

##下面定义一个线性模型

class LinearNet(nn.Module):         ##定义一个类 继承自nn.Module
    def __init__(self, n_feature):  ##创建类时定义的一个特殊方法__init__  只要创建该类的实例就会运行这个方法 此方法会在创建对象时完成初始化
                                    ##当构造函数被调用并创建一个新的对象时，self被设置为正在被初始化的对象
                                    ##在构造函数中创建任意的实例变量都可以保证在所有方法中都是可用的
        
        super(LinearNet, self).__init__() 
                                        ##super(Student,self).__init__()    这是对继承自父类的属性进行初始化。
                                        ##而且是用父类的初始化方法来初始化继承的属性。
                                        ##也就是说，子类继承了父类的所有属性和方法，父类属性自然会用父类方法来进行初始化。
                                        
                                
                                        ##Python中类的初始化都是__init__(), 所以父类和子类的初始化方式都是__init__(), 
                                        ##但是如果子类初始化时没有这个函数，那么他将直接调用父类的__init__(); 如果子类指定了__init__(), 
                                        ##就会覆盖父类的初始化函数__init__()，如果想在进行子类的初始化的同时也继承父类的__init__(), 
                                        ##就需要在子类中显示地通过super()来调用父类的__init__()函数。
                                    
        self.linear = nn.Linear(n_feature, 1)
        
                                        ##在类的代码（函数）中，访问当前的实例中的变量和函数首先需要实例对象本身
                                        ##而Python中就规定好了，函数的第一个参数，就必须是实例对象本身，并且建议，约定俗成，把其名字写为self
                                        ##而如果没有用到self，即代码中，去掉self后，那种写法所使用到的变量，实际上不是你所希望的，
                                        ##不是真正的实例中的变量和函数，而是的访问到了其他部分的变量和函数了。
                                        ##甚至会由于没有合适的初始化实例变量，而导致后续无法访问的错误。
                                        
    # forward 定义前向传播
    def forward(self, x):              ##定义一个类的普通方法（实例化方法），加self区别于普通函数
        
        y = self.linear(x)
        
        return y
                                        #实例方法隐含的参数为类实例self，而类方法隐含的参数为类本身cls。 
                                        #静态方法无隐含参数，主要为了类实例也可以直接调用静态方法。
                                        #逻辑上，类方法被类调用，实例方法被实例调用，静态方法两者都能调用。
                                        #主要区别在于参数传递上的区别，实例方法悄悄传递的是self引用作为参数，
                                        #而类方法悄悄传递的是cls引用作为参数。
                                        #类方法可以由类调用，而且因为传入了参数cls，故也可以由实例来调用
                                        #静态方法两者都能调用
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构


LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [5]:
#还可以用nn.Sequential来更加方便地搭建网络，Sequential是一个有序的容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中。
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
#net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])


Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [6]:
#可以通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器。
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.5290, -0.3215]], requires_grad=True)
Parameter containing:
tensor([-0.4744], requires_grad=True)


In [7]:
#作为一个单层神经网络，线性回归输出层中的神经元和输入层中各个输入完全连接。因此，线性回归的输出层又叫全连接层。

#注意：torch.nn仅支持输入一个batch的样本不支持单个样本输入，如果只有单个样本，可使用input.unsqueeze(0)来添加一维。

In [8]:
#在使用net前，我们需要初始化模型参数，如线性回归模型中的权重和偏差。
#PyTorch在init模块中提供了多种参数初始化方法。这里的init是initializer的缩写形式。
#我们通过init.normal_将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零。
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

#注：如果这里的net是用一开始的代码自定义的，那么上面代码会报错，
#net[0].weight应改为net.linear.weight，bias亦然。
#因为net[0]这样根据下标访问子模块的写法只有当net是个ModuleList或者Sequential实例时才可以

Parameter containing:
tensor([0.], requires_grad=True)

In [9]:
#PyTorch在nn模块中提供了各种损失函数，这些损失函数可看作是一种特殊的层，
#PyTorch也将这些损失函数实现为nn.Module的子类。我们现在使用它提供的均方误差损失作为模型的损失函数。
loss = nn.MSELoss()

In [10]:
#同样，我们也无须自己实现小批量随机梯度下降算法。
#torch.optim模块提供了很多常用的优化算法比如SGD、Adam和RMSProp等。
#下面我们创建一个用于优化net所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法。
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [11]:
#我们还可以为不同子网络设置不同的学习率，这在finetune时经常用到。例：
#optimizer =optim.SGD([
                # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                {'params': net.subnet1.parameters()}, # lr=0.03
#                {'params': net.subnet2.parameters(), 'lr': 0.01}
#            ], lr=0.03)

In [12]:
#有时候我们不想让学习率固定成一个常数，那如何调整学习率呢？
#主要有两种做法。一种是修改optimizer.param_groups中对应的学习率，
#另一种是更简单也是较为推荐的做法——新建优化器，由于optimizer十分轻量级，构建开销很小，故而可以构建新的optimizer。
#但是后者对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能会造成损失函数的收敛出现震荡等情况。

# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍


In [13]:
#在使用Gluon训练模型时，我们通过调用optim实例的step函数来迭代模型参数。
#按照小批量随机梯度下降的定义，我们在step函数中指明批量大小，从而对批量中样本梯度求平均。
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))


epoch 1, loss: 17.720951
epoch 2, loss: 4.250076
epoch 3, loss: 1.065902
epoch 4, loss: 0.373514
epoch 5, loss: 0.052752
epoch 6, loss: 0.033816
epoch 7, loss: 0.005902
epoch 8, loss: 0.002634
epoch 9, loss: 0.000621
epoch 10, loss: 0.000263


In [14]:
#下面我们分别比较学到的模型参数和真实的模型参数。我们从net获得需要的层，
#并访问其权重（weight）和偏差（bias）。学到的参数和真实的参数很接近。
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9942, -3.3909]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1883], requires_grad=True)
